```javascript
require('dotenv').config();

// create our express app
const express = require('express')
const app = express();

const connectDB = require('./config/database');

const PORT = process.env.PORT || 3000;

// ===== DATABASE CONNECTION =====
connectDB();
```
**We start by loading environment variables from a .env file using dotenv.**

**We import Express and create an Express application.**

**We import the connectDB function from ./config/database.**

**We set the port from the environment variable or default to 3000.**

**We call connectDB() to connect to the database.**

```javascript
// Express.json() middleware - parses incoming JSON requests
app.use(express.json());
app.use(express.urlencoded({extended: true}));
```
We set up middleware to parse JSON and urlencoded data (from forms).

```javascript
// ===== ROUTES =====
app.use('/api/books', require('./routes/bookRoutes'));
// app.use('/api/authors', require('./routes/authorRoutes')); // You'll create this
We set up routes for books and will do the same for authors.

```javascript
// ===== ROOT ROUTE =====
app.get('/', (req, res) => {
  res.json({
    message: '📚 Local Library API',
    version: '1.0.0',
    endpoints: {
      books: '/api/books',
      authors: '/api/authors',
      health: '/health'
    },
    timestamp: new Date().toISOString()
  });
});
```
We define a root route that returns a JSON response with API information.


```javascript
// ===== HEALTH CHECK =====
app.get('/health', (req, res) => {
  const mongoose = require('mongoose');
  const dbStatus = mongoose.connection.readyState;
  
  const statusMessages = {
    0: 'Disconnected',
    1: 'Connected',
    2: 'Connecting',
    3: 'Disconnecting'
  };

  res.status(200).json({
    status: 'OK',
    database: statusMessages[dbStatus] || 'Unknown',
    uptime: `${process.uptime().toFixed(2)} seconds`,
    memory: `${(process.memoryUsage().heapUsed / 1024 / 1024).toFixed(2)} MB`,
    timestamp: new Date().toISOString()
  });
});
```
We define a health check route that returns the status of the database and server.

```javascript
// ===== ERROR HANDLING =====

// Global error handler
app.use((error, req, res, next) => {
  console.error('Error:', error);

  // Mongoose validation error
  if (error.name === 'ValidationError') {
    const messages = Object.values(error.errors).map(val => val.message);
    return res.status(400).json({
      success: false,
      error: 'Validation Error',
      messages
    });
  }

  // Mongoose duplicate key error
  if (error.code === 11000) {
    return res.status(400).json({
      success: false,
      error: 'Duplicate field value entered'
    });
  }

  // Mongoose bad ObjectId
  if (error.name === 'CastError') {
    return res.status(400).json({
      success: false,
      error: 'Resource not found'
    });
  }

  res.status(500).json({
    success: false,
    error: 'Server Error',
    message: process.env.NODE_ENV === 'development' ? error.message : 'Internal server error'
  });
});
```
We set up a global error handler to catch and process errors.